In [17]:
import pickle
import numpy as np
import xgboost as xgb
import pandas as pd
from sklearn.utils import resample

In [4]:
with open("/kaggle/input/xgb-bestmodel-datathon2025/best_xgb_model.pkl", "rb") as file:
    model = pickle.load(file)

In [7]:
test_data = pd.read_csv("/kaggle/input/datathon2025/final_preprocessed_test.csv")
test_data.drop(columns=["Unnamed: 0"], inplace=True)
test_data

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,SaleType,SaleCondition,AgeWhenSold,RenovatedAgeSold
0,20,2.0,80.0,0.363804,1,0,3,3,0.0,4,...,-0.057207,0,3,0,-0.092212,6,8.0,4,49,49
1,20,3.0,81.0,0.897553,1,0,0,3,0.0,0,...,-0.057207,0,0,1,19.723675,6,8.0,4,52,52
2,60,3.0,74.0,0.809368,1,0,0,3,0.0,4,...,-0.057207,0,3,0,-0.092212,3,8.0,4,13,12
3,60,3.0,78.0,0.032053,1,0,0,3,0.0,4,...,-0.057207,0,0,0,-0.092212,6,8.0,4,12,12
4,120,3.0,43.0,-0.971475,1,0,0,1,0.0,4,...,-0.057207,0,0,0,-0.092212,1,8.0,4,18,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109,20,3.0,80.0,0.719368,1,0,3,3,0.0,4,...,-0.057207,0,0,0,-0.092212,5,8.0,4,37,27
1110,160,4.0,21.0,-1.672108,1,0,3,3,0.0,4,...,-0.057207,0,0,0,-0.092212,12,8.0,0,36,36
1111,160,4.0,21.0,-1.599260,1,0,3,3,0.0,4,...,-0.057207,0,0,0,-0.092212,4,8.0,0,36,36
1112,20,3.0,160.0,2.054445,1,0,3,3,0.0,4,...,-0.057207,0,0,0,-0.092212,9,8.0,0,46,10


In [8]:
predictions = model.predict(test_data)
predictions

array([130041.19, 156300.08, 196881.64, ...,  87467.28, 176599.23,
       255877.16], dtype=float32)

In [19]:
n_samples = 30 
bootstrap_preds = []

for _ in range(n_samples):
    sample_data = resample(test_data)  
    preds = model.predict(sample_data)
    bootstrap_preds.append(preds)

bootstrap_preds = np.array(bootstrap_preds)

# Compute mean and standard deviation
mean_predictions = bootstrap_preds.mean(axis=0)
std_predictions = bootstrap_preds.std(axis=0) 

confidence_percentage = 100 * (1 - (std_predictions / (std_predictions.max() + 1e-6)))  # Normalize

# Ensure confidence is between 0% and 100%
confidence_percentage = np.clip(confidence_percentage, 0, 100)

# Convert to DataFrame
results_df = pd.DataFrame({
    "Prediction": mean_predictions,
    "Confidence_Percentage": confidence_percentage
})

# Display results
print(results_df.head())

# Save predictions
results_df.to_csv("xgb_predictions_with_confidence_percentage.csv", index=False)

      Prediction  Confidence_Percentage
0  175342.937500              30.996222
1  187084.781250              26.544392
2  189493.156250              47.201836
3  178415.421875              36.930664
4  185689.468750              39.970009
